# Inport Libraries

In [13]:
import numpy as np
import math as mt
from datetime import *
import calendar
import netCDF4 as nc
import xarray as xr

In [14]:
input_folder = '/g/data/u46/users/ext547/ewater/input_data/test/'
min_nc = 'min_celsius_Tair_WFDEI_201612.nc'
max_nc = 'max_celsius_Tair_WFDEI_201612.nc'
mean_nc = 'mean_celsius_Tair_WFDEI_201612.nc'

# Input variables and reformat datatypes

In [15]:
min_xr = xr.open_dataset(input_folder+min_nc, decode_times=False)
mean_xr = xr.open_dataset(input_folder+mean_nc, decode_times=False)
max_xr = xr.open_dataset(input_folder+max_nc, decode_times=False)

In [24]:


time_index = 0
lat_index = 12
lon_index = 12

#Save time variable
time_unit = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
time_seconds = int(min_xr.time.isel(time=time_index).values)
date = time_unit + timedelta(seconds = time_seconds)
print('date: '+str(date))

#Save coordinates
lat_deg = float(min_xr.isel(time = time_index).lat[lat_index].values)
lon_deg = float(min_xr.isel(time = time_index).lon[lon_index].values)
print('lat: '+str(lat_deg))
print('lon: '+str(lon_deg))

#Save temperature varaibles
T_min =  float(min_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
T_mean = float(mean_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
T_max = float(max_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
print('Minimum temp: '+ str(T_min))
print('Mean temp: '+ str(T_mean))
print('Maximum temp: '+ str(T_max))

# Calculate temperature difference
# Units: T_diff in degrees C
# Example: T_diff = 34.8 - 25.6 = 9.2 degrees C

T_diff = T_max - T_min
print('T_DIFF: ' +str(T_diff))

# Convert latitude to radians
# Unit: lat_rad in radians (northern hemisphere)
# Example: lat_rad = (pi / 180) * 13.73 = + 0.240 radians

lat_rad = (mt.pi / 180) * lat_deg 

print('LAT_RAD'+str(lat_rad))

# Convert date to julian day of the year
# Unit: j_days in julian days
# Example: j_days = days between ("15/4/2015","1/1/2015") + 1 = 105 julian days

start_of_year = datetime.strptime('01/01/'+ str(date.year),'%m/%d/%Y') #save first day of year as variable
j_days = int(((date - start_of_year)+timedelta(days=1))/timedelta(days=1)) #calculate julian days
print('J_DAYS: '+str(j_days))

# Calculate the number of days in the year to account for leap year
# Units: ndy in days

if calendar.isleap(date.year) == True:
    print(str(date.year) +' is a leap year')
    ndy = 366
else:
    print(str(date.year) +' is not a leap year')
    ndy = 365
    

print('NDY: '+str(ndy))

# Caclulate the inverse relative distance between the Earth and the Sun
# Unit: dr in radians
# Example: dr = 1 + 0.033* cos(2*pi*105/365) = 0.992 radians

dr = 1 + 0.033 * mt.cos(2 * mt.pi * j_days / ndy)
print('DR: '+str(dr))

# Calculate solar declination
# Unit: sd in radians
# Example: sd = 0.409 * sin(2 * pi * 105 / 365 - 1.39) = 0.166 radians

sd = 0.409 * mt.sin(2 * mt.pi * 105 / 365 - 1.39)
print('SD: '+str(sd))

# Calculate a sunset angle input variable
# Unit: no unit
# Example: sa_X = 1 - [tan(0.24)]^2 * [tan(0.166)]^2 = 0.998

sa_X = 1 - (mt.tan(lat_rad))**2 * ((mt.tan(sd))**2)
print('SA_X: '+str(sa_X))

# Calculate sunset angle
# Units: sa in radians
# Example: sa = pi / 2 - arctan(-tan(0.24) * tan(0.166) / sqrt(0.998)) = 1.612 radians

sa = mt.pi / 2 - mt.atan(-mt.tan(lat_rad) * mt.tan(sd) / mt.sqrt(sa_X))
print('SA: '+str(sa))

# Calcualte extraterrestrial radiation
# Units: ra in MJ / (m^ day)
# Example: ra = 37.586 * 0.992 * (1.612 * sin(0.24) * sin(0.166) + cos(0.24) * cos(0.166) * sin(1.612)) = 38.06 MJ / (m^ day)

ra = 37.586 * dr * (sa * mt.sin(lat_rad) * mt.sin(sd) + mt.cos(lat_rad) * mt.cos(sd) * mt.sin(sa))
print('RA: '+str(ra))


# Calculate Potential Evapotranspiration
# Units: PET in mm/day
# Example: PET = 0.0009384 * (30.2 + 17.8) * sqrt(9.2) * 38.06 = 5.2 mm/day
pet = 0.0009384 * (T_mean + 17.8) * mt.sqrt(T_diff) * ra
print('PET: '+str(pet))


####Create array with values
#create an xarray
#create netcdf of PET


date: 2016-12-01 10:30:00
lat: 15.25
lon: 107.75
Minimum temp: 18.029809570312523
Mean temp: 19.518186187744163
Maximum temp: 21.868707275390648
T_DIFF: 3.838897705078125
LAT_RAD0.26616271092913524
J_DAYS: 336
2016 is a leap year
NDY: 366
DR: 1.0287194129539952
SD: 0.16583675497620104
SA_X: 0.9979177777858802
SA: 1.6164435505621821
RA: 39.467629079460025
PET: 2.7080260271549577


In [1]:
# #Save time variable
# time_unit = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
# time_seconds = int(min_xr.time.isel(time=time_index).values)
# date = time_unit + timedelta(seconds = time_seconds)
# print('date: '+str(date))

# #Save coordinates
# lat_deg = float(min_xr.isel(time = time_index).lat[lat_index].values)
# lon_deg = float(min_xr.isel(time = time_index).lon[lon_index].values)
# print('lat: '+str(lat_deg))
# print('lon: '+str(lon_deg))

# #Save temperature varaibles
# T_min =  float(min_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_mean = float(mean_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_max = float(max_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# print('Minimum temp: '+ str(T_min))
# print('Mean temp: '+ str(T_mean))
# print('Maximum temp: '+ str(T_max))

## Calculate max and min temperature difference

In [2]:
# # Calculate temperature difference
# # Units: T_diff in degrees C
# # Example: T_diff = 34.8 - 25.6 = 9.2 degrees C

# T_diff = T_max - T_min
# print(T_diff)

## Convert latitude

In [3]:
# # Convert latitude to radians
# # Unit: lat_rad in radians (northern hemisphere)
# # Example: lat_rad = (pi / 180) * 13.73 = + 0.240 radians

# lat_rad = (mt.pi / 180) * lat_deg 

# print(lat_rad)

## Calculate days and dates

In [4]:
# # Convert date to julian day of the year
# # Unit: j_days in julian days
# # Example: j_days = days between ("15/4/2015","1/1/2015") + 1 = 105 julian days

# start_of_year = datetime.strptime('01/01/'+ str(date.year),'%m/%d/%Y') #save first day of year as variable
# j_days = int(((date - start_of_year)+timedelta(days=1))/timedelta(days=1)) #calculate julian days
# print(j_days)

In [5]:
# # Calculate the number of days in the year to account for leap year
# # Units: ndy in days

# if calendar.isleap(date.year) == True:
#     print(str(date.year) +' is a leap year')
#     ndy = 366
# else:
#     print(str(date.year) +' is not a leap year')
#     ndy = 365
    

# print('There are '+str(ndy)+' days')

## Calculate extraterrestrial radiation

In [6]:
# # Caclulate the inverse relative distance between the Earth and the Sun
# # Unit: dr in radians
# # Example: dr = 1 + 0.033* cos(2*pi*105/365) = 0.992 radians

# dr = 1 + 0.033 * mt.cos(2 * mt.pi * j_days / ndy)
# print (dr)

In [7]:
# # Calculate solar declination
# # Unit: sd in radians
# # Example: sd = 0.409 * sin(2 * pi * 105 / 365 - 1.39) = 0.166 radians

# sd = 0.409 * mt.sin(2 * mt.pi * 105 / 365 - 1.39)
# print(sd)

In [8]:
# # Calculate a sunset angle input variable
# # Unit: no unit
# # Example: sa_X = 1 - [tan(0.24)]^2 * [tan(0.166)]^2 = 0.998

# sa_X = 1 - (mt.tan(lat_rad))**2 * ((mt.tan(sd))**2)
# print(sa_X)

In [10]:
# # Calculate sunset angle
# # Units: sa in radians
# # Example: sa = pi / 2 - arctan(-tan(0.24) * tan(0.166) / sqrt(0.998)) = 1.612 radians

# sa = mt.pi / 2 - mt.atan(-mt.tan(lat_rad) * mt.tan(sd) / mt.sqrt(sa_X))
# print(sa)

In [9]:
# # Calcualte extraterrestrial radiation
# # Units: ra in MJ / (m^ day)
# # Example: ra = 37.586 * 0.992 * (1.612 * sin(0.24) * sin(0.166) + cos(0.24) * cos(0.166) * sin(1.612)) = 38.06 MJ / (m^ day)

# ra = 37.586 * dr * (sa * mt.sin(lat_rad) * mt.sin(sd) + mt.cos(lat_rad) * mt.cos(sd) * mt.sin(sa))
# print(ra)

## Calculate Potential Evapotranspiration

In [11]:
# # Calculate Potential Evapotranspiration
# # Units: PET in mm/day
# # Example: PET = 0.0009384 * (30.2 + 17.8) * sqrt(9.2) * 38.06 = 5.2 mm/day
# pet = 0.0009384 * (T_mean + 17.8) * mt.sqrt(T_diff) * ra
# print(pet)

In [11]:
# #Input variables
# #latitude (in degrees and minutes)
# input_lat = '13 44 N'
# #date (in mm/dd/yyyy)
# input_date = '04/15/2015'
# #daily maximum temperature (degrees C)
# input_Tmax =  '34.8'
# #daily mean temperature (degrees C)
# input_Tmean = '30.2'
# #daily minimum temperature (degrees C)
# input_Tmin = '25.6'

In [12]:
# #Reformat datatypes
# lat_deg = int(input_lat.split(' ')[0])
# lat_min = int(input_lat.split(' ')[1])
# T_max= float(input_Tmax)
# T_mean = float(input_Tmean)
# T_min = float(input_Tmin)
# date = datetime.strptime(input_date,'%m/%d/%Y') #convert to datetime format
# year = date.year #save year as variable

In [14]:
#Convert latitude to decimal degree
#Unit: lat_deg in decimal degrees
#Example: lat_deg = 13 + 44 / 60 = 13.73 decimal degrees

# lat_deg = lat_deg + lat_min/int(60)

# print(lat_deg)